# Manejo de tablas con `dplyr`  
Este Notebook muestra alguna de las funciones más comunes del paquete dplyr, explicando su sintaxis y mostando algunos ejemplos de usos.

In [2]:
#Instalar paquetes la primera vez:
#install.packages("tidyverse")
#install.packages("dslabs")

In [2]:
library(tidyverse)
library(dslabs)

## Tidyverse

Tidyverse agrupa un conjunto de paquetes que contienen funciones para el tratamiento y visualizacion de datos en R:  
- dplyr = manejo de tablas de datos 
- ggplot2 = creación de gráficos de alta calidad y personalizables
- readr = lectura de archivos con tablas de datos
- tibble = creación y manejo de tablas tipo *tibble* (una version mejorada de los data frames básicos)
- stringr = operaciones con *strings* (texto)
- tidyr = manupulación de tablas para hacelas _alargadas_ (agrupar las variables, pocas columnas y muchas observaciones), o _ensanchadas_ (proceso contrario)
- forcats = operaciones para manejo de variables categóricas
- purrr = aplicar operaciones a cada elemento de una columna o fila (entre otras cosas)

Tidyverse está muy extendido como el principal entorno para manejo de tablas de datos. Todos los paquetes comparten una sintaxis y un funcionamiento similares que permiten realizar las tareas con un codigo mas limpio, entendible y sin incompatibilidades.  

En este notebook se mostarán principalmente las funcionalidades de la librería **dplyr**, aunque complementando con alguna otra (p.ej. tidyr)

## dslabs / gapminder

dslabs es un paquete que cuenta con diversos datasets a los que se puede llamar directamente usando la librería. Estos datasets están especialmente pensados para ser usados en pruebas y tutoriales, con tablas de datos ya organizados de diferentes temáticas y estructuras  

En este caso los ejemplos usarán la tabla gapminder: un juego de palabras con el aviso que suena por megafonia en el metro de londres ("mind the gap"), y al mismo tiempo sobre las desigualdades ("gap") que hay entre paises en diferentes partes del mundo, y cómo han ido cambiando con el tiempo.

El impulsor de estos datos es Hans Rosling, que en [esta maravillosa charla TED](https://www.youtube.com/watch?v=hVimVzgtD6w) cambió la forma no solo de ver las diferencias entre primer/tercer mundo, si no tambien de lo que todo lo pueden contar los datos y lo importante de tenerlos accesibles.

In [3]:
df <- gapminder
head(df)
tail(df)

,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
1,Albania,1960,115.40,62.87,6.19,1636054,NA,Europe,Southern Europe
2,Algeria,1960,148.20,47.50,7.65,11124892,13828152297,Africa,Northern Africa
3,Angola,1960,208.00,35.98,7.32,5270844,NA,Africa,Middle Africa
4,Antigua and Barbuda,1960,NA,62.97,4.43,54681,NA,Americas,Caribbean
5,Argentina,1960,59.87,65.39,3.11,20619075,108322326649,Americas,South America
6,Armenia,1960,NA,66.86,4.55,1867396,NA,Asia,Western Asia


,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
10540,Venezuela,2016,NA,74.80,NA,NA,NA,Americas,South America
10541,West Bank and Gaza,2016,NA,74.70,NA,NA,NA,Asia,Western Asia
10542,Vietnam,2016,NA,75.60,NA,NA,NA,Asia,South-Eastern Asia
10543,Yemen,2016,NA,64.92,NA,NA,NA,Asia,Western Asia
10544,Zambia,2016,NA,57.10,NA,NA,NA,Africa,Eastern Africa
10545,Zimbabwe,2016,NA,61.69,NA,NA,NA,Africa,Eastern Africa


## El pipe ( %>% )

Uno de los elementos más distintivos de tidyverse es el pipe (escrito como %>% ) 

Normalmente la forma de realizar varias operaciones seguidas en R es ejecutar una por línea (sobreescribiendo el objeto o creando varios), o bien anidando funciones a base de meter una funcion dentro de otra que a su vez está dentro de otra (con el riesgo de errores y confusión)  

Para ello, con tidyverse suele emplearse el pipe, que permite encadenar las funciones a ejecutar en fila, de forma que el resultado saliente de una entre directamente en la siguiente, consiguiendo un código más limpio y legible.

In [3]:
#Calcular la media de una columna y redondear
#Metodo clásico
round(mean(df$life_expectancy))

[1] 65

In [4]:
#Usando el pipe
df$life_expectancy %>% mean() %>% round()

[1] 65

Y por supuesto, tambien se puede guardar como objeto (da igual al principio o al final)

In [6]:
mean_life_expectancy <- df$life_expectancy %>% mean() %>% round()

df$life_expectancy %>% mean() %>% round() -> mean_life_expectancy_alt 

Al comparar si son iguales:

In [7]:
mean_life_expectancy == mean_life_expectancy_alt

[1] TRUE

## Operaciones básicas

**Nota:** cuando estamos trabajando con funciones de tidyverse sobre un dataframe, se puede llamar a las columnas directamente, sin necesidad de usar '$'

### Select (seleccionar columnas)

Sintaxis básica:  
**select**(el `dataframe` (entra con el pipe), `nombre de columna/s`  con las que quedarse)

In [8]:
#viene bien recordar los nombres de las columnas
names(df)

[1] "country"          "year"             "infant_mortality" "life_expectancy" 
[5] "fertility"        "population"       "gdp"              "continent"       
[9] "region"

In [9]:
#Seleccionar solo algunas columnas
df %>% select(country, continent, region) %>% head()

,country,continent,region
,<fct>,<fct>,<fct>
1,Albania,Europe,Southern Europe
2,Algeria,Africa,Northern Africa
3,Angola,Africa,Middle Africa
4,Antigua and Barbuda,Americas,Caribbean
5,Argentina,Americas,South America
6,Armenia,Asia,Western Asia


In [10]:
#Seleccionar por exclusión de otras (todas menos "continent" y "region")
df %>% select(-continent, -region) %>% head()

,country,year,infant_mortality,life_expectancy,fertility,population,gdp
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Albania,1960,115.40,62.87,6.19,1636054,NA
2,Algeria,1960,148.20,47.50,7.65,11124892,13828152297
3,Angola,1960,208.00,35.98,7.32,5270844,NA
4,Antigua and Barbuda,1960,NA,62.97,4.43,54681,NA
5,Argentina,1960,59.87,65.39,3.11,20619075,108322326649
6,Armenia,1960,NA,66.86,4.55,1867396,NA


### Filter (filtrar filas)

Una función muy util que permite quedarte solo con las filas (casos) que cumplen un determinado criterio, similar al proceso de hacer un query en otros lenguajes

**Sintaxis básica:**

filter(el data frame, nombre de columna == 'criterio que cumplen las filas que te interesan')

El criterio puede ser cualquier comparador o que devuelva TRUE/FALSE, por ejemplo:  
- **==** que sea igual a un valor o palabra
- **!=** que sea diferente a un valor o palabra
- **>** **<** que sea mayor o menor que un valor
- **>=** **=<** que sea igual o mayor/menor a un valor
- **is.na()** que sea NA 
- **!is.na()** que NO sea NA 

Además se pueden encadenar argumentos:  
- **&**: condición 1 **y** condicion 2
- **|**: condición 1 **o** condicion 2

In [10]:
#Selección sencilla
df %>% filter(country == 'France') %>% head()

country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
France,1960,23.7,70.49,2.77,45865699,349778187326,Europe,Western Europe
France,1961,22.4,71.07,2.80,46471083,369037927246,Europe,Western Europe
France,1962,21.3,70.61,2.81,47121575,393660782189,Europe,Western Europe
France,1963,20.3,70.46,2.81,47781535,414710508267,Europe,Western Europe
France,1964,19.4,71.43,2.80,48402900,441742742172,Europe,Western Europe
France,1965,18.5,71.26,2.77,48952283,462849365785,Europe,Western Europe


In [24]:
#Selección por dos criterios
df %>% filter(country == 'Spain' & year >= 2008)

country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
Spain,2008,4.2,81.1,1.42,45817016,740341770865,Europe,Southern Europe
Spain,2009,4.1,81.5,1.44,46295191,712647416141,Europe,Southern Europe
Spain,2010,3.9,81.8,1.46,46601492,712152291146,Europe,Southern Europe
Spain,2011,3.8,82.0,1.47,46708366,717193989759,Europe,Southern Europe
Spain,2012,3.7,82.2,1.49,46637082,NA,Europe,Southern Europe
Spain,2013,3.7,82.5,1.51,46455163,NA,Europe,Southern Europe
Spain,2014,3.6,82.5,1.52,46259716,NA,Europe,Southern Europe
Spain,2015,3.5,82.6,1.53,46121699,NA,Europe,Southern Europe
Spain,2016,NA,82.7,NA,NA,NA,Europe,Southern Europe


In [11]:
#Seleccionar que los NO tengan NAs en un campo (y ver solo los primeros resultados)
df %>% filter(!is.na(gdp)) %>% head()

,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
1,Algeria,1960,148.20,47.50,7.65,11124892,13828152297,Africa,Northern Africa
2,Argentina,1960,59.87,65.39,3.11,20619075,108322326649,Americas,South America
3,Australia,1960,20.30,70.87,3.45,10292328,96677859364,Oceania,Australia and New Zealand
4,Austria,1960,37.30,68.75,2.70,7065525,52392699681,Europe,Western Europe
5,Bahamas,1960,51.00,62.00,4.50,109526,1306269490,Americas,Caribbean
6,Bangladesh,1960,176.30,46.20,6.73,48200702,12767231590,Asia,Southern Asia


###Sampling (muestreo aleatorio)
Las funciones de sampleo permiten seleccionar aleatoriamente registros de una tabla, bien sea por cantidad con `sample_n()` o por proporción con `sample_frac()`.  

**Sintaxis básica:**  
sample_X(`dataframe`, `cantidad/proporción`, `replace = TRUE/FALSE`)  

El parametro `replace` indica si un mismo registro se puede seleccionar mas de una vez (replace = True) o si se debe samplear sin repetición (replace = False)

In [10]:
# Seleccionar 10 obsevaciones al azar

set.seed(123) # Aqui se fija una "semilla" para generar numeros random. Si la semilla es la misma, el resultado también
df %>% sample_n(10, replace = FALSE)

country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
France,1973,13.1,72.69,2.24,52214014,705840360681,Europe,Western Europe
Maldives,1973,152.0,49.98,7.10,127717,NA,Asia,Southern Asia
French Polynesia,2016,NA,76.91,NA,NA,NA,Oceania,Polynesia
Brazil,2007,17.1,73.00,1.94,192784521,815703390474,Americas,South America
Burkina Faso,1976,131.9,43.74,6.92,6274037,1003222368,Africa,Western Africa
Uruguay,1969,49.6,68.84,2.87,2795044,11891487796,Americas,South America
Jamaica,2010,15.5,74.80,2.33,2741253,9799664035,Americas,Caribbean
Cote d'Ivoire,1978,120.2,51.70,7.77,7569558,8475427578,Africa,Western Africa
Puerto Rico,1985,NA,74.00,2.34,3369714,31189568233,Americas,Caribbean


### Arrange (ordenar por una columna)

Arrange ordena todo el dataframe en función de los valores (numéricos, alfabéticos o de orden de factor) de una columna seleccionada.  

**Sintaxis básica:**  
arrange(el `dataframe`, `columna de ordenación`) 

nota: Por defecto, la ordenación es ascendente (de menor a mayor). Para una ordenación descendente, se incluiría el nombre de la columna dentro de la función `desc()`, de la siguiente manera:  
arrange(el `dataframe`, desc(`columna de ordenación`))

In [5]:
#Ver los 10 años de España con mayor esperanza de vida
#Filtrar por pais = España, ordenar por esperanza de vida descendente, ver 10 primeros registros
df %>% filter(country == 'Spain') %>% arrange(desc(life_expectancy)) %>% head()

,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
1,Spain,2016,NA,82.7,NA,NA,NA,Europe,Southern Europe
2,Spain,2015,3.5,82.6,1.53,46121699,NA,Europe,Southern Europe
3,Spain,2013,3.7,82.5,1.51,46455163,NA,Europe,Southern Europe
4,Spain,2014,3.6,82.5,1.52,46259716,NA,Europe,Southern Europe
5,Spain,2012,3.7,82.2,1.49,46637082,NA,Europe,Southern Europe
6,Spain,2011,3.8,82.0,1.47,46708366,7.17194e+11,Europe,Southern Europe


### Mutate (crear columnas)

Mutate permite crear nuevas columnas aplicando un criterio (normalmente transformaciones de una o varias columnas ya existentes)

**Sintaxis básica:**  
mutate(el `dataframe`, `Nueva_columna` = `Operacion para generarla`)

In [15]:
#Calcular el PIB per capita a partir de dividir la columna gdp (gross domestic product) entre la de population
#Nota: para eliminar  primero las filas que tengan ALGUN valor NA (en cualquier columna), se usa na.omit()
df %>% mutate(gdp_per_capapita = gdp/population) %>% na.omit() %>%  head()

,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region,gdp_per_capapita
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>
2,Algeria,1960,148.20,47.50,7.65,11124892,13828152297,Africa,Northern Africa,1242.9920
5,Argentina,1960,59.87,65.39,3.11,20619075,108322326649,Americas,South America,5253.5008
8,Australia,1960,20.30,70.87,3.45,10292328,96677859364,Oceania,Australia and New Zealand,9393.1965
9,Austria,1960,37.30,68.75,2.70,7065525,52392699681,Europe,Western Europe,7415.2593
11,Bahamas,1960,51.00,62.00,4.50,109526,1306269490,Americas,Caribbean,11926.5699
13,Bangladesh,1960,176.30,46.20,6.73,48200702,12767231590,Asia,Southern Asia,264.8765


### Case_when (generar valores según múltiples condiciones)

La función case_when es muy util cuando se quiere, por ejemplo, crear una nueva variable que sea el resultado de aplicar varios argumentos condicionales. Por ejemplo, si queremos que nos añada un valor o una etiqueta dependiendo de si se cumple una u otra condición.  

**Sintaxis**

La sintaxis de case_when puede parecer inicialmente más complicada, pero una vez entendida resulta sencilla de usar y ampliar los casos.

de forma basica, sería:
case_when(`condición` ~ `resultado`)

En la condición se incluye un conjunto de datos (p.ej. una columna de una tabla), y el resultado de la función será una lista de datos de la misma longitud, los cuales mostrarán el `resultado` en los casos en los que se cumpla la `condición` impuesta.

Ejemplo: con los datos de 2011, ver qué paises serían considerados como desarrollados ( _developed_ ) o en vías de desarrollo ( _developing_ ) en base a si el cálculo de PIB per capita es > 12000 (developed) o menor (developing)

In [19]:
# Crear tabla filtrada con datos solo de 2011
df_2011 <- df %>% filter(year == 2011)

In [26]:
case_when((df_2011$gdp / df_2011$population) >= 12000 ~ "DEVELOPED",
         (df_2011$gdp / df_2011$population) <= 12000 ~ "Developing")

[1] "Developing" "Developing" "Developing" "Developing" "Developing"
  [6] "Developing" NA           "DEVELOPED"  "DEVELOPED"  "Developing"
 [11] "DEVELOPED"  NA           "Developing" NA           "Developing"
 [16] "DEVELOPED"  "Developing" "Developing" "Developing" "Developing"
 [21] "Developing" "Developing" "Developing" NA           "Developing"
 [26] "Developing" "Developing" "Developing" "Developing" "DEVELOPED" 
 [31] "Developing" "Developing" "Developing" "Developing" "Developing"
 [36] "Developing" "Developing" "Developing" "Developing" "Developing"
 [41] "Developing" "Developing" NA           "Developing" "Developing"
 [46] "DEVELOPED"  NA           "Developing" "Developing" "Developing"
 [51] "Developing" "Developing" "Developing" "Developing" "Developing"
 [56] "Developing" "DEVELOPED"  "DEVELOPED"  NA           "Developing"
 [61] "Developing" "Developing" "DEVELOPED"  "Developing" "DEVELOPED" 
 [66] NA           "Developing" "Developing" "Developing" "Developing"
 [71] NA           "Developing" "Developing" "DEVELOPED"  "Developing"
 [76] "DEVELOPED"  "Developing" "Developing" NA           "Developing"
 [81] "DEVELOPED"  "DEVELOPED"  "DEVELOPED"  "Developing" "DEVELOPED" 
 [86] "Developing" "Developing" "Developing" "Developing" "DEVELOPED" 
 [91] "DEVELOPED"  "Developing" "Developing" "Developing" "Developing"
 [96] "Developing" "Developing" NA           "Developing" "DEVELOPED" 
[101] "DEVELOPED"  "Developing" "Developing" "Developing" "Developing"
[106] "Developing" "Developing" "Developing" "Developing" "Developing"
[111] "Developing" "Developing" "Developing" "Developing" "Developing"
[116] "Developing" "Developing" "Developing" "Developing" "DEVELOPED" 
[121] NA           NA           "Developing" "Developing" "Developing"
[126] "DEVELOPED"  "Developing" "Developing" "Developing" "Developing"
[131] "Developing" "Developing" "Developing" "Developing" "Developing"
[136] NA           "DEVELOPED"  "Developing" "Developing" "Developing"
[141] "Developing" "Developing" "Developing" "Developing" "Developing"
[146] "Developing" "Developing" "Developing" "DEVELOPED"  "Developing"
[151] "DEVELOPED"  "Developing" "Developing" "DEVELOPED"  "Developing"
[156] "Developing" NA           "Developing" "DEVELOPED"  "DEVELOPED" 
[161] NA           "Developing" "Developing" "Developing" "Developing"
[166] "Developing" "Developing" "Developing" "Developing" "Developing"
[171] "Developing" "Developing" "Developing" "DEVELOPED"  "DEVELOPED" 
[176] "DEVELOPED"  "Developing" "Developing" "Developing" "Developing"
[181] NA           "Developing" "Developing" "Developing" "Developing"

Este resultado tiene 2 problemas:  
1) Una lista como tal no es útil, es mejor si se asigna a una nueva variable de la tabla.  
2) Hay casos que se escapan a las condiciones descritas (los NAs, por ejemplo).

Para ello, vamos a guardar el resultado como una nueva variable  **STATUS** en la tabla, pero esta vez definiendo como developing cpuntries a los que tengan un resultado > 5000, y indicando dos nuevas condiciones.  
Primero, que los resultados que sean NA los marque como "No info".  
Segundo que los casos que no caigan dentro de ninguna de las condiciones se les asigne la etiqueta "other" (en este caso serían las economías del tercer mundo). Indicamos esos "casos restantes sin contemplar" utilizando como último argumento _TRUE_


**Nota:** las condiciones en case_when se aplican en el orden escrito, es decir, si un caso cumple la primera, ya no será considerado en las siguientes.

In [30]:
df_2011$STATUS <- case_when((df_2011$gdp / df_2011$population) >= 12000 ~ "DEVELOPED",
                       (df_2011$gdp / df_2011$population) >= 5000 ~ "Developing",
                        is.na(df_2011$gdp / df_2011$population) ~ "No information",
                        TRUE ~ "Other")

head(df_2011)
table(df_2011$STATUS) #para ver un conteo de cuantos casos hay de cada tipo

,country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region,STATUS
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<chr>
1,Albania,2011,14.3,77.4,1.75,2886010,6321690864,Europe,Southern Europe,Other
2,Algeria,2011,22.8,76.1,2.83,36717132,81143448101,Africa,Northern Africa,Other
3,Angola,2011,106.8,58.1,6.10,21942296,27013935821,Africa,Middle Africa,Other
4,Antigua and Barbuda,2011,7.2,75.9,2.12,88152,801787943,Americas,Caribbean,Developing
5,Argentina,2011,12.7,76.0,2.20,41655616,472935255184,Americas,South America,Developing
6,Armenia,2011,15.3,73.5,1.50,2967984,4290990647,Asia,Western Asia,Other



     DEVELOPED     Developing No information          Other 
            31             29             17            108 

### Un poco de repaso...
Teniendo las funciones principales, vamos a concatenar unas cuantas.

Ejemplo: paises europeos con mayor PIB per capita en el año 2011  
Para ello, en una sola linea continua, vamos a...
- seleccionar solo nombre de pais, continente, PIB (GDP) y población
- filtrar solo los que correspondan al año y la region
- calcular la nueva columna con el valor de PIB per capita
- sacar un ranking solo con las 10 con mayor valor de esa estadística (funcion top_n(), mezcla de sample_n() y arrange() )

In [17]:
df2 <- df %>% select(c(country, year, population, gdp, continent)) %>% #seleccion de columnas
             filter(continent == 'Europe' & year == 2011) %>% #filtrar casos
             mutate(gdp_percap = gdp/population) %>% #crear nueva columna
             top_n(10, gdp_percap) #top 10 ordenado por gdp_percap
df2

country,year,population,gdp,continent,gdp_percap
<fct>,<int>,<dbl>,<dbl>,<fct>,<dbl>
Austria,2011,8423559,2.309133e+11,Europe,27412.80
Denmark,2011,5576577,1.710519e+11,Europe,30673.27
Finland,2011,5395816,1.495734e+11,Europe,27720.26
Iceland,2011,321030,1.110516e+10,Europe,34592.27
Luxembourg,2011,519981,2.692859e+10,Europe,51787.64
Netherlands,2011,16689863,4.463622e+11,Europe,26744.51
Norway,2011,4953945,1.982526e+11,Europe,40019.13
Sweden,2011,9462352,3.167986e+11,Europe,33479.90
Switzerland,2011,7925813,3.009384e+11,Europe,37969.41


## Group by (agrupar)

group_by permite agrupar las observaciones en funcion de una variable de interés, y calcular estadísticas en cada grupo (media, numero de casos, etc...)

Para ello, el procedimiento cuenta de 2 pasos: 

1) generar una tabla agrupada con la funcion **group_by**, que en apariencia será igual que la original (pero ya tiene el criterio de agrupamiento como una propiedad "oculta") 

2) extraer un resumen o tabla agrupada con la funcion **summarise**, donde se aplica una **función de agregación** que nos devolverá una variable "resumen" con la información que queramos calculada para cada grupo

In [20]:
#Agrupar los datos por cada año
#resumir la tabla al promedio de esperanza de vida en gada grupo (año)
df %>% group_by(year) %>% 
    summarise("Avg life expectancy" = mean(life_expectancy))

year,Avg life expectancy
<int>,<dbl>
1960,54.89211
1961,55.39892
1962,55.84605
1963,56.36076
1964,56.87789
1965,57.25757
1966,57.68438
1967,58.12005
1968,58.47870


In [25]:
#Seleccionar solo los paises europeos
#agrupar los datos de todo el periodo por cada pais
#y crear una tabla resumiendo el promedio de PIB en cada pais para todos los años (sin contar NAs)

df %>% filter(continent == 'Europe') %>%
    group_by(country) %>%
    summarise('Mean GDP' = mean(gdp, na.rm = TRUE)) %>% 
    head(10)

country,Mean GDP
<fct>,<dbl>
Albania,3720794204
Austria,136429756535
Belarus,15948205130
Belgium,169790462909
Bosnia and Herzegovina,5884484391
Bulgaria,14527469868
Croatia,23264163380
Czech Republic,63891609013
Denmark,116892788898


#### Summarise_each

### Joins (uniones de tablas)

### Reshapes (cambiar la distribución de la tabla)

Pasar filas a columnas o viceversa